In [ ]:
# 사용된 특수 문자 추출

import os
import re
import pandas as pd

# 특수문자만 추출하는 정규 표현식 패턴 (한글, 영어, 숫자, 한자, 일본어 제외)
special_char_pattern = r'[^가-힣a-zA-Z0-9一-龥ぁ-ゔァ-ヴー々〆〤\s]'

# 폴더 경로와 결과 저장할 엑셀 파일 이름 설정
folder_path = r'경로'  # 폴더 경로로 변경
output_path = '파일명.xlsx'

# 특수문자를 저장할 집합 (중복 제거용)
unique_special_chars = set()

# 폴더 안의 모든 엑셀 파일을 읽어서 특수문자 추출
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        file_path = os.path.join(folder_path, filename)
        try:
            # 엑셀 파일 읽기
            xl = pd.ExcelFile(file_path)
            for sheet_name in xl.sheet_names:
                df = xl.parse(sheet_name)
                
                # 모든 셀에 대해 특수문자 추출
                for col in df.columns:
                    df[col] = df[col].astype(str)  # 모든 값을 문자열로 변환
                    for cell in df[col]:
                        special_chars = re.findall(special_char_pattern, cell)
                        unique_special_chars.update(special_chars)  # 집합에 추가하여 중복 제거
        except Exception as e:
            print(f"파일 {filename}에서 오류 발생: {e}")

# 추출한 특수문자를 데이터프레임으로 변환 후 엑셀 파일로 저장
extracted_df = pd.DataFrame({'특수문자': list(unique_special_chars)})
extracted_df.to_excel(output_path, index=False)

print("특수문자 추출 및 중복 제거가 완료되었습니다.")

In [ ]:
# 헷갈리는 특수문자 앞뒤 문맥 추출

import os
import re
import pandas as pd

# 폴더 경로 설정
folder_path = r'경로'  # 폴더 경로로 변경
output_path = '파일명.xlsx'

# 특수문자 목록과 패턴 (정규식 사용을 위해 이스케이프 문자 필요)
special_chars = ['〃']
special_char_pattern = '|'.join([re.escape(char) for char in special_chars])

# 추출 결과 저장을 위한 리스트
extracted_data = []

# 폴더 내 모든 엑셀 파일 처리
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        file_path = os.path.join(folder_path, filename)
        try:
            # 엑셀 파일 읽기
            xl = pd.ExcelFile(file_path)
            for sheet_name in xl.sheet_names:
                df = xl.parse(sheet_name)
                
                # 모든 셀에 대해 특수문자 앞뒤 어절 추출
                for col in df.columns:
                    df[col] = df[col].astype(str)  # 모든 값을 문자열로 변환
                    for cell in df[col]:
                        # 특수문자 위치 찾기
                        matches = re.finditer(special_char_pattern, cell)
                        for match in matches:
                            start = max(0, match.start() - 100)  # 특수문자 앞 어절 시작점
                            end = min(len(cell), match.end() + 100)  # 특수문자 뒤 어절 끝점
                            context = cell[start:end]  # 앞뒤 어절 포함한 부분 추출
                            extracted_data.append({
                                '파일명': filename,
                                '시트명': sheet_name,
                                '열': col,
                                '특수문자': match.group(),
                                '문맥': context
                            })
        except Exception as e:
            print(f"파일 {filename}에서 오류 발생: {e}")

# 결과를 데이터프레임으로 변환 후 엑셀 파일로 저장
extracted_df = pd.DataFrame(extracted_data)
extracted_df.to_excel(output_path, index=False)

print("특수문자 앞뒤 문맥 추출이 완료되었습니다.")


In [ ]:
# 작품명 관련 특수 문자 정규화. 동아일보는 되지 않음.

import os
import pandas as pd
import re

# 폴더 경로 설정
folder_path = r'경로'

# 대체할 특수문자 매핑 (왼쪽 문자: 오른쪽 문자)
replace_dict = {
    "'": "＇",
    '"': '"',
    "＂": '"',
    "(": "(",
    "（": "(",
    ")": ")",
    "）": ")",
    "[": "[",
    "［": "[",
    "]": "]",
    "］": "]",
    "`": "＇", 
    "｀": "＇",
    "{": "{",
    "}": "}",
    "´": "＇",
    "˝": '"',
    "‘": "＇",
    "’": "＇",
    "‛": "＇",
    "“": '"',
    "”": '"',
    "′": "＇",
    "″": '"',
    "〈": "<",
    "〉": ">",
    "《": "《",
    "》": "》",
    "｢": "｢",
    "「": "｢",
    "｣": "｣",
    "」": "｣",
    "『": "󰡔",
    "』": "󰡕",
    "【": "【",
    "】": "】",
    "〔": "[",
    "〕": "]",
    "<": "<",
    "＜": "<",
    ">": ">",
    "＞": ">",
    "≪": "《",
    "≫": "》",
    "󰡔": "󰡔",
    "󰡕": "󰡕",
}

# 오류 로그 기록
error_log = []

# 폴더 내 모든 엑셀 파일 처리
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls') or filename.startswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        
        try:
            # 엑셀 파일 열기
            xl = pd.ExcelFile(file_path, engine='openpyxl')
            
            with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                for sheet_name in xl.sheet_names:
                    try:
                        # 시트 읽기 및 빈 데이터 확인
                        df = xl.parse(sheet_name)
                        if df.empty:
                            print(f"{filename} - 시트 '{sheet_name}'이 비어 있습니다. 건너뜁니다.")
                            continue
                        
                        # 모든 셀에 대해 대체 문자 적용
                        for col in df.columns:
                            try:
                                df[col] = df[col].astype(str).replace("nan", "")
                                for old_char, new_char in replace_dict.items():
                                    try:
                                        # 대체 적용 및 오류 처리
                                        df[col] = df[col].str.replace(re.escape(old_char), new_char, regex=True)
                                    except Exception as cell_error:
                                        error_log.append(f"'{filename}', 시트 '{sheet_name}', 열 '{col}'에서 '{old_char}' 대체 중 오류: {cell_error}")
                                        continue
                            except Exception as col_error:
                                error_log.append(f"'{filename}', 시트 '{sheet_name}', 열 '{col}' 처리 중 오류: {col_error}")
                                continue
                        
                        # 변경 내용을 파일에 저장
                        df.to_excel(writer, sheet_name=sheet_name, index=False)
                        print(f"{filename} - 시트 '{sheet_name}' 처리 완료.")

                    except Exception as sheet_error:
                        error_log.append(f"'{filename}' 파일의 시트 '{sheet_name}' 처리 중 오류: {sheet_error}")
                        continue

        except PermissionError:
            error_log.append(f"'{filename}' 파일에서 권한 오류 발생: 다른 프로그램에서 파일이 열려 있을 수 있습니다.")
        except Exception as e:
            error_log.append(f"'{filename}' 파일 열기 중 오류 발생: {e}")

print("처리가 완료되었습니다. 다음은 발생한 오류 목록입니다(있을 경우):")
for error in error_log:
    print(error)

In [ ]:
# 특수문자 정규화 200행씩 분할. 동아일보

import os
import pandas as pd
import re

# 폴더 경로 설정 (예시 경로)
folder_path = r'경로'

replace_dict = {
    "'": "＇",
    '"': '"',
    "＂": '"',
    "(": "(",
    "（": "(",
    ")": ")",
    "）": ")",
    "[": "[",
    "［": "[",
    "]": "]",
    "］": "]",
    "`": "＇",   
    "｀": "＇",
    "{": "{",
    "}": "}",
    "´": "＇",
    "˝": '"',
    "‘": "＇",
    "’": "＇",
    "‛": "＇",
    "“": '"',
    "”": '"',
    "′": "＇",
    "″": '"',
    "〈": "<",
    "〉": ">",
    "《": "《",
    "》": "》",
    "｢": "｢",
    "「": "｢",
    "｣": "｣",
    "」": "｣",
    "『": "󰡔",
    "』": "󰡕",
    "【": "【",
    "】": "】",
    "〔": "[",
    "〕": "]",
    "<": "<",
    "＜": "<",
    ">": ">",
    "＞": ">",
    "≪": "《",
    "≫": "》",
    "󰡔": "󰡔",
    "󰡕": "󰡕",
}

# 병합할 데이터프레임 초기화
final_merged_df = pd.DataFrame()

# 오류 로그
error_log = []

# 각 파일에 대해 처리
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls') or filename.startswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        
        try:
            xl = pd.ExcelFile(file_path, engine='openpyxl')
            
            for sheet_name in xl.sheet_names:
                try:
                    df = xl.parse(sheet_name)
                    if df.empty:
                        print(f"{filename} - 시트 '{sheet_name}'이 비어 있습니다. 건너뜁니다.")
                        continue
                    
                    # 200행씩 분할하여 처리
                    num_splits = (len(df) // 200) + 1
                    for i in range(num_splits):
                        split_df = df.iloc[i * 200 : (i + 1) * 200].copy()

                        # 각 셀에 대체 문자 적용
                        for col in split_df.columns:
                            try:
                                split_df.loc[:, col] = split_df[col].astype(str).replace("nan", "")
                                for old_char, new_char in replace_dict.items():
                                    try:
                                        split_df.loc[:, col] = split_df[col].str.replace(re.escape(old_char), new_char, regex=True)
                                    except Exception as cell_error:
                                        error_log.append(f"'{filename}', 시트 '{sheet_name}', 열 '{col}'에서 '{old_char}' 대체 중 오류: {cell_error}")
                                        continue
                            except Exception as col_error:
                                error_log.append(f"'{filename}', 시트 '{sheet_name}', 열 '{col}' 처리 중 오류: {col_error}")
                                continue

                        # 처리된 200행을 최종 병합 데이터프레임에 추가
                        final_merged_df = pd.concat([final_merged_df, split_df], ignore_index=True)

                except Exception as sheet_error:
                    error_log.append(f"'{filename}' 파일의 시트 '{sheet_name}' 처리 중 오류: {sheet_error}")
                    continue

        except PermissionError:
            error_log.append(f"'{filename}' 파일에서 권한 오류 발생: 다른 프로그램에서 파일이 열려 있을 수 있습니다.")
        except Exception as e:
            error_log.append(f"'{filename}' 파일 열기 중 오류 발생: {e}")

# 모든 처리가 끝난 후 최종 병합된 파일 저장
merged_file_path = os.path.join(folder_path, "final_merged_output.xlsx")
final_merged_df.to_excel(merged_file_path, index=False)
print(f"최종 병합 파일이 '{merged_file_path}'에 저장되었습니다.")

# 오류 로그 출력
print("처리가 완료되었습니다. 다음은 발생한 오류 목록입니다(있을 경우):")
for error in error_log:
    print(error) 



In [ ]:
# 특수문자 사이 단어 추출

import pandas as pd
import re
import glob
import os

# 처리할 엑셀 파일들이 있는 폴더 경로 지정
folder_path = r'경로'

# 폴더 내 모든 .xlsx 파일 검색
excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))

# 지정된 특수 문자 쌍 정의
special_char_pairs = [
    ("＇", "＇"), 
    ('"', '"'), 
    ("(", ")"), 
    ("[", "]"), 
    ("{", "}"), 
    ("<", ">"), 
    ("《", "》"), 
    ("｢", "｣"), 
    ("󰡔", "󰡕"), 
    ("【", "】")
]

# 각 특수 문자 쌍에 대한 정규 표현식 패턴 생성
pattern = re.compile('|'.join([re.escape(o) + r'(.+?)' + re.escape(c) for o, c in special_char_pairs]))

# 추출한 결과를 저장할 세트 (중복 제거)
extracted_data = set()

# 에러 로그 기록용 리스트
error_log = []

for file in excel_files:
    try:
        print(f'{file} 처리 중...')
        
        # 엑셀 파일의 모든 시트 읽기
        xls = pd.ExcelFile(file)
        for sheet_name in xls.sheet_names:
            try:
                df = pd.read_excel(file, sheet_name=sheet_name, dtype=str)
                
                # 모든 셀에서 특수 문자 쌍 포함 텍스트 추출
                for col in df.columns:
                    for cell in df[col].dropna():  # 비어있지 않은 셀에 대해서만 처리
                        # 정규표현식으로 특수문자 쌍 포함 추출
                        matches = pattern.finditer(cell)
                        
                        # 각 매칭 결과를 세트에 추가 (중복 제거)
                        for match in matches:
                            extracted_data.add(match.group(0))  # 특수문자 포함 전체 문자열 추가
            
            except Exception as sheet_error:
                error_log.append(f'파일: {file}, 시트: {sheet_name} 처리 중 에러 발생 - {sheet_error}')
                print(f'파일: {file}, 시트: {sheet_name} 처리 중 에러 발생 - {sheet_error}')

    except Exception as file_error:
        error_log.append(f'파일: {file} 열기 중 에러 발생 - {file_error}')
        print(f'파일: {file} 열기 중 에러 발생 - {file_error}')

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame(sorted(extracted_data), columns=["Extracted Text"])

# 시트 최대 행수인 1,048,576 행을 초과하지 않도록 나누어 저장
output_file = '특수문자_사이_단어_포함.xlsx'
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for i in range(0, len(result_df), 1048576):
        result_df.iloc[i:i+1048576].to_excel(writer, sheet_name=f'Sheet_{i // 1048576 + 1}', index=False)

# 에러 로그 출력
if error_log:
    print("\n에러 로그:")
    for log in error_log:
        print(log)
else:
    print("모든 파일이 성공적으로 처리되었습니다.")

print(f'추출 완료! 결과는 {output_file} 파일을 확인하세요.')


In [ ]:
# 한자 띄어쓰기, 공백 삭제

import os
import pandas as pd
import regex as re

# 연속된 한자 사이의 공백과 줄바꿈 제거 함수
def remove_spaces_and_line_breaks(text):
    if isinstance(text, str):
        # 연속된 한자를 찾고 그 사이의 공백과 줄바꿈을 제거
        new_text = re.sub(r'(\p{IsHan})\s+(?=\p{IsHan})', r'\1', text)
        if new_text != text:
            print(f"변경 전: {text}")
            print(f"변경 후: {new_text}\n")
        return new_text
    return text

# 줄바꿈을 공백으로 대체하는 함수 (content 열에 적용)
def replace_line_breaks_with_spaces(text):
    if isinstance(text, str):
        # 모든 줄바꿈을 공백으로 대체
        text = text.replace('\n', ' ')
    return text

# Excel 파일들을 처리하는 메인 함수
def process_excel_files(folder_path):
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
            file_path = os.path.join(folder_path, file_name)
            df = pd.read_excel(file_path)
            
            # 'title', 'subtitle', 'content' 열에 연속된 한자 처리
            for column in ['title', 'subtitle', 'content']:
                if column in df.columns:
                    df[column] = df[column].apply(remove_spaces_and_line_breaks)
            
            # 'content' 열에 줄바꿈을 공백으로 대체
            if 'content' in df.columns:
                df['content'] = df['content'].apply(replace_line_breaks_with_spaces)
            
            # 결과를 저장 (원본을 덮어쓰거나 다른 이름으로 저장할 수 있습니다)
            df.to_excel(file_path, index=False)
            
# 사용할 폴더 경로를 여기에 설정하세요.
folder_path = r'경로'
process_excel_files(folder_path)


In [ ]:
# 주요 한자어 변환(글자수 많은 것부터 적은 것 순으로 변경)

import os
import pandas as pd
import re

# 폴더 경로 설정
folder_path = r'경로'

# 대체할 특수문자 매핑 (왼쪽 문자: 오른쪽 문자)
replace_dict = {
#     r"高\s*等\s*學\s*校": "고등학교",    r"萬\s*里\s*長\s*城": "만리장성",    r"文\s*化\s*勳\s*章": "문화훈장",    r"不\s*信\s*時\s*代": "불신시대",    r"서\s*울\s*大\s*學": "서울대학",
#     r"修\s*文\s*書\s*館": "수문서관",    r"藥\s*으\s*로\s*도": "약으로도",    r"梨\s*花\s*女\s*大": "이화여대",    r"海\s*東\s*旅\s*館": "해동여관",    r"黑\s*黑\s*白\s*白": "흑흑백백",
#     r"暗\s*黑\s*時\s*代": "암흑시대",    r"延\s*世\s*大\s*學": "연세대학",

#     r"姜\s*萬\s*吉": "강만길",    r"敎\s*科\s*書": "교과서",    r"金\s*東\s*里": "김동리",    r"金\s*玲\s*珠": "김영주",    r"金\s*藥\s*局": "김약국",
#     r"金\s*芝\s*河": "김지하",    r"金\s*治\s*洙": "김치수",    r"金\s*炯\s*國": "김형국",    r"綠\s*地\s*帶": "녹지대",    r"大\s*學\s*校": "대학교",
#     r"渡\s*船\s*場": "도선장",    r"柳\s*宗\s*鎬": "유종호",    r"李\s*御\s*寧": "이어령",    r"李\s*鍾\s*桓": "이종환",    r"文\s*學\s*賞": "문학상",
#     r"文\s*化\s*社": "문화사",    r"民\s*音\s*社": "민음사",    r"密\s*告\s*者": "밀고자",    r"朴\s*景\s*利": "박경리",    r"朴\s*今\s*伊": "박금이",
#     r"朴\s*婉\s*緖": "박완서",    r"朴\s*花\s*城": "박화성",    r"白\s*樂\s*晴": "백낙청",    r"汎\s*友\s*社": "범우사",    r"思\s*想\s*界": "사상계",
#     r"瑞\s*文\s*堂": "서문당",    r"서\s*울\s*大": "서울대",    r"産\s*業\s*社": "산업사",    r"新\s*太\s*陽": "신태양",    r"實\s*技\s*論": "실기론",
#     r"雙\s*頭\s*兒": "쌍두아",    r"延\s*世\s*大": "연세대",    r"藝\s*術\s*賞": "예술상",    r"仁\s*村\s*賞": "인촌상",    r"日\s*本\s*論": "일본론",
#     r"再\s*歸\s*熱": "재귀열",    r"全\s*光\s*鏞": "전광용",    r"鄭\s*麟\s*永": "정인영",    r"鄭\s*顯\s*琦": "정현기",    r"地\s*理\s*山": "지리산",
#     r"靑\s*少\s*年": "청소년",    r"崔\s*一\s*男": "최일남",    r"出\s*版\s*社": "출판사",    r"平\s*面\s*圖": "평면도",    r"平\s*沙\s*里": "평사리",
#     r"漂\s*流\s*島": "표류도",    r"玄\s*岩\s*社": "현암사",    r"湖\s*巖\s*賞": "호암상",    r"天\s*道\s*敎": "천도교",    r"女\s*學\s*生": "여학생",

    # r"家\s*庭": "가정",    r"決\s*定": "결정",    r"京\s*鄕": "경향",    r"計\s*算": "계산",    r"公\s*論": "공론",
    #     r"敎\s*授": "교수",    r"國\s*際": "국제",    r"貴\s*族": "귀족",    r"克\s*日": "극일",    r"紀\s*念": "기념",
    # r"金\s*冠": "금관",    r"女\s*流": "여류",    r"女\s*史": "여사",    r"女\s*士": "여사",    r"女\s*苑": "여원",
    #     r"女\s*人": "여인",    r"單\s*層": "단층",    r"短\s*篇": "단편",    r"大\s*衆": "대중",    r"大\s*河": "대하",
    # r"大\s*學": "대학",    r"大\s*韓": "대한",    r"圖\s*表": "도표",    r"導\s*標": "도표",    r"道\s*標": "도표",
    #     r"東\s*光": "동광",    r"東\s*民": "동민",    r"東\s*亞": "동아",    r"童\s*話": "동화",    r"來\s*成": "내성",    
    # r"戀\s*歌": "연가",    r"連\s*載": "연재",    r"聯\s*合": "연합",    r"玲\s*珠": "영주",    r"魔\s*女": "마녀",
    #     r"晩\s*洲": "만주",    r"滿\s*珠": "만주",    r"名\s*譽": "명예",    r"木\s*蓮": "목련",    r"文\s*壇": "문단",    
    # r"文\s*明": "문명",    r"文\s*藝": "문예",    r"文\s*人": "문인",    r"文\s*學": "문학",    r"文\s*化": "문화",
    #     r"迷\s*那": "미나",    r"民\s*族": "민족",    r"民\s*主": "민주",    r"博\s*士": "박사",    r"伴\s*侶": "반려",    
    # r"反\s*日": "반일",    r"白\s*鐵": "백철",    r"僻\s*地": "벽지",    r"寶\s*冠": "보관",    r"釜\s*山": "부산",
    #     r"不\s*安": "불안",    r"夫\s*人": "부인",    r"思\s*想": "사상",    r"散\s*考": "산고",    r"三\s*省": "삼성",    
    # r"生\s*命": "생명",    r"生\s*活": "생활",    r"書\s*閣": "서각",    r"先\s*生": "선생",    r"聖\s*女": "성녀",
    #     r"世\s*界": "세계",    r"世\s*代": "세대",    r"小\s*說": "소설",    r"小\s*話": "소화",    r"速\s*斷": "속단",    
    # r"受\s*賞": "수상",    r"隨\s*筆": "수필",    r"淑\s*蘭": "숙란",    r"淑\s*明": "숙명",    r"宿\s*題": "숙제",
    #     r"時\s*期": "시기",    r"時\s*代": "시대",    r"時\s*報": "시보",    r"市\s*場": "시장",    r"市\s*井": "시정",    
    # r"食\s*口": "식구",    r"新\s*聞": "신문",    r"新\s*報": "신보",    r"新\s*銳": "신예",    r"新\s*人": "신인",
    #     r"新\s*作": "신작",    r"蟋\s*蟀": "실솔",    r"失\s*人": "부인",    r"十\s*五": "십오",    r"樂\s*士": "악사",    
    # r"暗\s*黑": "암흑",    r"哀\s*歌": "애가",    r"女\s*高": "여고",    r"女\s*流": "여류",    r"女\s*像": "여상",
    #     r"女\s*性": "여성",    r"女\s*子": "여자",    r"歷\s*史": "역사",    r"宴\s*歌": "연가",    r"延\s*大": "연대",    
    # r"永\s*遠": "영원",    r"映\s*畵": "영화",    r"午\s*後": "오후",    r"外\s*廓": "외곽",    r"寓\s*話": "우화",
    #     r"運\s*動": "운동",    r"運\s*河": "운하",    r"原\s*作": "원작",    r"原\s*州": "원주",    r"月\s*刊": "월간",    
    # r"月\s*灘": "월탄",    r"銀\s*河": "은하",    r"乙\s*酉": "을유",    r"梨\s*花": "이화",    r"日\s*報": "일보",
    #     r"日\s*本": "일본",    r"日\s*月": "일월",    r"日\s*帝": "일제",    r"自\s*由": "자유",    r"作\s*家": "작가",    
    # r"作\s*品": "작품",    r"雜\s*誌": "잡지",    r"掌\s*篇": "장편",    r"長\s*篇": "장편",    r"再\s*婚": "재혼",
    #     r"著\s*者": "저자",    r"全\s*南": "전남",    r"剪\s*刀": "전도",    r"展\s*望": "전망",    r"戰\s*場": "전장",    
    # r"戰\s*後": "전후",    r"政\s*經": "정경",    r"貞\s*陵": "정릉",    r"精\s*銳": "정예",    r"正\s*午": "정오",
    #     r"帝\s*國": "제국",    r"條\s*件": "조건",    r"朝\s*鮮": "조선",    r"罪\s*人": "죄인",    r"主\s*婦": "주부",    
    # r"主\s*義": "주의",    r"中\s*國": "중국",    r"中\s*央": "중앙",    r"中\s*日": "중일",    r"地\s*帶": "지대",
    #     r"知\s*性": "지성",    r"知\s*識": "지식",    r"晉\s*州": "진주",    r"創\s*作": "창작",    r"忠\s*武": "충무",    
    # r"他\s*人": "타인",    r"土\s*地": "토지",    r"通\s*信": "통신",    r"統\s*營": "통영",    r"波\s*市": "파시",
    #     r"評\s*論": "평론",    r"風\s*景": "풍경",    r"河\s*東": "하동",    r"韓\s*國": "한국",    r"合\s*唱": "합창",    
    # r"現\s*代": "현대",    r"湖\s*水": "호수",    r"湖\s*巖": "호암",    r"環\s*境": "환경",    r"幻\s*想": "환상",
    #     r"悔\s*悟": "회오",    r"勳\s*章": "훈장",    r"薰\s*香": "훈향",    r"讀\s*者": "독자",    r"讀\s*書": "독서",    
    # r"全\s*集": "전집",    r"詩\s*集": "시집",    r"書\s*店": "서점",    r"出\s*版": "출판",    r"革\s*命": "혁명",
    #     r"東\s*學": "동학",    r"農\s*民": "농민",    r"協\s*會": "협회",    r"雪\s*花": "설화",    r"敎\s*育": "교육",

    # "家": "가",    "間": "간",    "官": "관",    "校": "교",    "郡": "군",    "洞": "동",    "病": "병",    "社": "사",    "水": "수",    "市": "시",
    # "新": "신",    "申": "신",    "氏": "씨",    "人": "인",    "者": "자",    "族": "족",    "窓": "창",    "集": "집",    "史": "사",    "詩": "시",
    # "金藥局": "김약국", "女性": "여성", "歷": "역", "同": "동", "女大生": "여대생", "會": "회", "朴景利": "박경리", "梨大": "이대", "梨花": "이화", "小說": "소설",
    # "女苑": "여원", "女像": "여상",
#
#  }

# 정규화까지 한 다음에 할 것.

"가정생활(가정생활)": "가정생활", "가톨릭신문(가톨릭신문)": "가톨릭신문", "강만길(강만길)": "강만길", "경향신문(경향신문)": "경향신문", "계산(계산)": "계산",
"국제신보(국제신보)": "국제신보", "귀족(귀족)": "귀족", "극일(극일)": "극일", "금관문화훈장(금관문화훈장)": "금관문화훈장", "기념(기념)": "기념",
"김동리(김동리)": "김동리", "김약국(김약국)": "김약국", "김영주(김영주)": "김영주", "김지하(김지하)": "김지하", "김치수(김치수)": "김치수",
"김형국(김형국)": "김형국", "내성문학상(내성문학상)": "내성문학상", "녹지대(녹지대)": "녹지대", "단층(단층)": "단층", "단편소설(단편소설)": "단편소설",
"대중(대중)": "대중", "대하소설(대하소설)": "대하소설", "대학생(대학생)": "대학생", "대한교과서(대한교과서)": "대한교과서", "도선장(도선장)": "도선장",
"도표(도표)": "도표", "동광(동광)": "동광", "동광출판사(동광출판사)": "동광출판사", "동민(동민)": "동민", "동민문화사(동민문화사)": "동민문화사",
"동아일보(동아일보)": "동아일보", "동화(동화)": "동화", "마녀(마녀)": "마녀", "만리장성(만리장성)": "만리장성", "만주(만주)": "만주",
"목련(목련)": "목련", "문단(문단)": "문단", "문명(문명)": "문명", "문예영화(문예영화)": "문예영화", "문인(문인)": "문인",
"문학(문학)": "문학", "문학과지성(문학과지성)": "문학과지성", "문학사상(문학사상)": "문학사상", "문화(문화)": "문화",
"문화일보(문화일보)": "문화일보", "미나(미나)": "미나", "민음사(민음사)": "민음사", "민족(민족)": "민족", "민족주의(민족주의)": "민족주의",
"민주신보(민주신보)": "민주신보", "밀고자(밀고자)": "밀고자", "박경리(박경리)": "박경리", "박경리기념관(박경리기념관)": "박경리기념관",
"박경리문학공원(박경리문학공원)": "박경리문학공원", "박경리문학관(박경리문학관)": "박경리문학관", "박경리문학상(박경리문학상)": "박경리문학상", "박경리문학제(박경리문학제)": "박경리문학제", "박경리씨(박경리씨)": "박경리씨",
"박금이(박금이)": "박금이", "박완서(박완서)": "박완서", "박화성(박화성)": "박화성", "반려(반려)": "반려", "반일(반일)": "반일",
"백낙청(백낙청)": "백낙청", "백철(백철)": "백철", "범우사(범우사)": "범우사", "벽지(벽지)": "벽지", "병(병)": "병",
"보관문화훈장(보관문화훈장)": "보관문화훈장", "부산일보(부산일보)": "부산일보", "부인(부인)": "부인", "불신시대(불신시대)": "불신시대", "불안(불안)": "불안",
"사상계(사상계)": "사상계", "삼성출판사(삼성출판사)": "삼성출판사", "생명(생명)": "생명", "서문당(서문당)": "서문당", "서울대학교(서울대학교)": "서울대학교",
"선생(선생)": "선생", "성녀(성녀)": "성녀", "세계(세계)": "세계", "세대(세대)": "세대", "소설(소설)": "소설",
"소설가(소설가)": "소설가", "소설문학(소설문학)": "소설문학", "소설집(소설집)": "소설집", "소설토지학교(소설토지학교)": "소설토지학교", "속단(속단)": "속단",
"수문서관(수문서관)": "수문서관", "수상(수상)": "수상", "수필(수필)": "수필", "수필집(수필집)": "수필집", "숙란(숙란)": "숙란",
"숙제(숙제)": "숙제", "시경소화(시경소화)": "시경소화", "시기(시기)": "시기", "시장(시장)": "시장", "시집(시집)": "시집",
"식구(식구)": "식구", "식솔(식솔)": "식솔", "신교수(신교수)": "신교수", "신동아(신동아)": "신동아", "신예(신예)": "신예",
"신작십오인집(신작십오인집)": "신작십오인집", "신태양(신태양)": "신태양", "신태양사(신태양사)": "신태양사", "쌍두아(쌍두아)": "쌍두아", "씨(씨)": "씨",
"악사(악사)": "악사", "암흑시대(암흑시대)": "암흑시대", "애가(애가)": "애가", "약(약)": "약", "여대생(여대생)": "여대생",
"여류(여류)": "여류", "여사(여사)": "여사", "여상(여상)": "여상", "여성(여성)": "여성", "여성동아(여성동아)": "여성동아",
"여원(여원)": "여원", "여인(여인)": "여인", "역사소설(역사소설)": "역사소설", "연가(연가)": "연가", "연세대학교(연세대학교)": "연세대학교",
"연재소설(연재소설)": "연재소설", "영원(영원)": "영원", "영주(영주)": "영주", "오후(오후)": "오후", "외곽지대(외곽지대)": "외곽지대",
"우화(우화)": "우화", "운하(운하)": "운하", "원작(원작)": "원작", "원주(원주)": "원주", "원주시(원주시)": "원주시",
"원주통신(원주통신)": "원주통신", "월간경항(월간경항)": "월간경항", "월간문학(월간문학)": "월간문학", "월간중앙(월간중앙)": "월간중앙", "월탄문학상(월탄문학상)": "월탄문학상",
"유종호(유종호)": "유종호", "은하(은하)": "은하", "은하수(은하수)": "은하수", "을유문화사(을유문화사)": "을유문화사", "이어령(이어령)": "이어령",
"이종환(이종환)": "이종환", "이화여자대학교(이화여자대학교)": "이화여자대학교", "인간(인간)": "인간", "인촌상(인촌상)": "인촌상", "일본(일본)": "일본",
"일본론(일본론)": "일본론", "일본산고(일본산고)": "일본산고", "일본인(일본인)": "일본인", "일본제국(일본제국)": "일본제국", "일월서각(일월서각)": "일월서각",
"일제(일제)": "일제", "자유공론(자유공론)": "자유공론", "작가(작가)": "작가", "작품(작품)": "작품", "잡지(잡지)": "잡지",
"장편소설(장편소설)": "장편소설", "재귀열(재귀열)": "재귀열", "재혼(재혼)": "재혼", "저자(저자)": "저자", "전광용(전광용)": "전광용",
"전남일보(전남일보)": "전남일보", "전도(전도)": "전도", "전망(전망)": "전망", "전장(전장)": "전장", "정경문화(정경문화)": "정경문화",
"정릉(정릉)": "정릉", "정릉동(정릉동)": "정릉동", "정오(정오)": "정오", "정인영(정인영)": "정인영", "정현기(정현기)": "정현기",
"조건(조건)": "조건", "조선일보(조선일보)": "조선일보", "족(족)": "족", "죄인(죄인)": "죄인", "주부생활(주부생활)": "주부생활",
"중국(중국)": "중국", "중앙일보(중앙일보)": "중앙일보", "중일(중일)": "중일", "지리산(지리산)": "지리산", "지식산업사(지식산업사)": "지식산업사",
"진주여자고등학교(진주여자고등학교)": "진주여자고등학교", "창(창)": "창", "창작실기론(창작실기론)": "창작실기론", "청소년(청소년)": "청소년", "최일남(최일남)": "최일남",
"충무(충무)": "충무", "타인(타인)": "타인", "토지(토지)": "토지", "토지문학제(토지문학제)": "토지문학제", "토지문화관(토지문화관)": "토지문화관",
"토지문화재단(토지문화재단)": "토지문화재단", "토지사랑회(토지사랑회)": "토지사랑회", "통영(통영)": "통영", "통영시(통영시)": "통영시", "파시(파시)": "파시",
"평면도(평면도)": "평면도", "평사리(평사리)": "평사리", "표류도(표류도)": "표류도", "풍경(풍경)": "풍경", "하동(하동)": "하동",
"하동군(하동군)": "하동군", "한국문학(한국문학)": "한국문학", "한국여류문학상(한국여류문학상)": "한국여류문학상", "한국여류문학인회(한국여류문학인회)": "한국여류문학인회", "한국일보(한국일보)": "한국일보",
"한국평론(한국평론)": "한국평론", "합창(합창)": "합창", "해동여관(해동여관)": "해동여관", "현대문학(현대문학)": "현대문학", "현대문학사(현대문학사)": "현대문학사",
"현대문학상(현대문학상)": "현대문학상", "현암사(현암사)": "현암사", "호수(호수)": "호수", "호암상(호암상)": "호암상", "호암상예술상(호암상예술상)": "호암상예술상",
"호암예술상(호암예술상)": "호암예술상", "환경운동연합(환경운동연합)": "환경운동연합", "환상(환상)": "환상", "회오(회오)": "회오", "훈향(훈향)": "훈향",
"흑흑백백(흑흑백백)": "흑흑백백", "영화(영화)":"영화", "박경리(박경리)":"박경리"
}




target_columns = ["title", "sub-title", "content"]

# 오류 로그 기록
error_log = []

# 폴더 내 모든 엑셀 파일 처리
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        file_path = os.path.join(folder_path, filename)
        
        try:
            # 엑셀 파일 열기
            xl = pd.ExcelFile(file_path, engine='openpyxl')
            
            with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                for sheet_name in xl.sheet_names:
                    try:
                        # 시트 읽기 및 빈 데이터 확인
                        df = xl.parse(sheet_name)
                        if df.empty:
                            print(f"{filename} - 시트 '{sheet_name}'이 비어 있습니다. 건너뜁니다.")
                            continue
                        
                        # 특정 열만 처리
                        for col in target_columns:
                            if col in df.columns:
                                try:
                                    df[col] = df[col].fillna("")
                                    for old_char, new_char in replace_dict.items():
                                        try:
                                            # 대체 적용
                                            df[col] = df[col].str.replace(old_char, new_char, regex=True)
                                        except Exception as cell_error:
                                            error_log.append(f"'{filename}', 시트 '{sheet_name}', 열 '{col}'에서 '{old_char}' 대체 중 오류: {cell_error}")
                                            continue
                                except Exception as col_error:
                                    error_log.append(f"'{filename}', 시트 '{sheet_name}', 열 '{col}' 처리 중 오류: {col_error}")
                                    continue
                        
                        # 변경 내용을 파일에 저장
                        df.to_excel(writer, sheet_name=sheet_name, index=False)
                        print(f"{filename} - 시트 '{sheet_name}' 처리 완료.")

                    except Exception as sheet_error:
                        error_log.append(f"'{filename}' 파일의 시트 '{sheet_name}' 처리 중 오류: {sheet_error}")
                        continue

        except PermissionError:
            error_log.append(f"'{filename}' 파일에서 권한 오류 발생: 다른 프로그램에서 파일이 열려 있을 수 있습니다.")
        except Exception as e:
            error_log.append(f"'{filename}' 파일 열기 중 오류 발생: {e}")

print("처리가 완료되었습니다. 다음은 발생한 오류 목록입니다(있을 경우):")
for error in error_log:
    print(error)

In [ ]:
pip install regex